Below is code with a link to a happy or sad dataset which contains 80 images, 40 happy and 40 sad. 
Create a convolutional neural network that trains to 100% accuracy on these images,  which cancels training upon hitting training accuracy of >.999

Hint -- it will work best with 3 convolutional layers.

In [1]:
import tensorflow as tf
import os
import zipfile

In [2]:
DESIRED_ACCURACY = 0.999

!wget --no-check-certificate \
    "https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip" \
    -O "/tmp/happy-or-sad.zip"

zip_ref = zipfile.ZipFile("/tmp/happy-or-sad.zip", 'r')
zip_ref.extractall("/tmp/h-or-s")
zip_ref.close()

--2019-06-23 13:18:29--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip
Resolving storage.googleapis.com... 216.58.195.80
Connecting to storage.googleapis.com|216.58.195.80|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2670333 (2.5M) [application/zip]
Saving to: ‘/tmp/happy-or-sad.zip’

/tmp/happy-or-sad.z 100%[===================>]   2.55M   172KB/s    in 13s     

2019-06-23 13:18:42 (206 KB/s) - ‘/tmp/happy-or-sad.zip’ saved [2670333/2670333]



In [12]:
class myCallback(tf.keras.callbacks.Callback):
    
    def on_epoch_end(self, epoch, logs={}):
        if DESIRED_ACCURACY <= logs.get('acc'):
            self.model.stop_training = True
            print("\nReached 99.9% accuracy so cancelling training!")
callbacks = myCallback()

In [14]:
# This Code Block should Define and Compile the Model
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3, 3), input_shape=(150, 150, 3), activation='relu'),
    tf.keras.layers.MaxPool2D(2, 2),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPool2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPool2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

from tensorflow.keras.optimizers import RMSprop

model.compile(optimizer=RMSprop(lr=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [6]:
# This code block should create an instance of an ImageDataGenerator called train_datagen 
# And a train_generator by calling train_datagen.flow_from_directory

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1/255)

train_generator = train_datagen.flow_from_directory(
    '/tmp/h-or-s',
    target_size=(150, 150),
    batch_size=40,
    class_mode='binary'
    )

# Expected output: 'Found 80 images belonging to 2 classes'

Found 80 images belonging to 2 classes.


In [15]:
# This code block should call model.fit_generator and train for
# a number of epochs. 
history = model.fit_generator(
    train_generator,
    steps_per_epoch=2,
    epochs=20,
    verbose=1,
    callbacks=[callbacks]
    )
    
# Expected output: "Reached 99.9% accuracy so cancelling training!""

Epoch 1/20
2/2 [==============================] - 3s 1s/step - loss: 0.7203 - acc: 0.5000
Epoch 2/20
2/2 [==============================] - 2s 923ms/step - loss: 0.7884 - acc: 0.4750
Epoch 3/20
2/2 [==============================] - 2s 952ms/step - loss: 0.6609 - acc: 0.6875
Epoch 4/20
2/2 [==============================] - 2s 918ms/step - loss: 0.5387 - acc: 0.8500
Epoch 5/20
2/2 [==============================] - 2s 1s/step - loss: 0.9030 - acc: 0.5250
Epoch 6/20
1/2 [==============>...............] - ETA: 0s - loss: 0.5555 - acc: 1.0000
Reached 99.9% accuracy so cancelling training!
2/2 [==============================] - 2s 919ms/step - loss: 0.4917 - acc: 1.0000
